In [85]:
import os
import cv2
import numpy as np
import math
import datetime
import argparse
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator, Colors
from copy import deepcopy

sport_list = {
    'deadlift': {
        'left_points_idx': [11, 13, 15],
        'right_points_idx': [12, 14, 16],
        'maintaining': 167,
        'relaxing': 177
    },
    'bench': {
        'left_points_idx': [5, 7, 9],
        'right_points_idx': [6, 8, 10],
        'maintaining': 70,
        'relaxing': 110
    },
    'squat': {
        'left_points_idx': [11, 13, 15],
        'right_points_idx': [12, 14, 16],
        'maintaining': 85,
        'relaxing': 140
    }
}


def calculate_angle(key_points, left_points_idx, right_points_idx):
    def _calculate_angle(line1, line2):
        # Calculate the slope of two straight lines
        slope1 = math.atan2(line1[3] - line1[1], line1[2] - line1[0])
        slope2 = math.atan2(line2[3] - line2[1], line2[2] - line2[0])

        # Convert radians to angles
        angle1 = math.degrees(slope1)
        angle2 = math.degrees(slope2)

        # Calculate angle difference
        angle_diff = abs(angle1 - angle2)

        # Ensure the angle is between 0 and 180 degrees
        if angle_diff > 180:
            angle_diff = 360 - angle_diff

        return angle_diff

    left_points = [[key_points.data[0][i][0], key_points.data[0][i][1]] for i in left_points_idx]
    right_points = [[key_points.data[0][i][0], key_points.data[0][i][1]] for i in right_points_idx]
    line1_left = [
        left_points[1][0].item(), left_points[1][1].item(),
        left_points[0][0].item(), left_points[0][1].item()
    ]
    line2_left = [
        left_points[1][0].item(), left_points[1][1].item(),
        left_points[2][0].item(), left_points[2][1].item()
    ]
    angle_left = _calculate_angle(line1_left, line2_left)
    line1_right = [
        right_points[1][0].item(), right_points[1][1].item(),
        right_points[0][0].item(), right_points[0][1].item()
    ]
    line2_right = [
        right_points[1][0].item(), right_points[1][1].item(),
        right_points[2][0].item(), right_points[2][1].item()
    ]
    angle_right = _calculate_angle(line1_right, line2_right)
    angle = (angle_left + angle_right) / 2
    return angle


def plot(pose_result, plot_size_redio, show_points=None, show_skeleton=None):
    class _Annotator(Annotator):

        def kpts(self, kpts, shape=(640, 640), radius=5, line_thickness=2, kpt_line=True):
            if self.pil:
                # Convert to numpy first
                self.im = np.asarray(self.im).copy()
            nkpt, ndim = kpts.shape
            is_pose = nkpt == 17 and ndim == 3
            kpt_line &= is_pose  # `kpt_line=True` for now only supports human pose plotting
            colors = Colors()
            for i, k in enumerate(kpts):
                if show_points is not None:
                    if i not in show_points:
                        continue
                color_k = [int(x) for x in self.kpt_color[i]] if is_pose else colors(i)
                x_coord, y_coord = k[0], k[1]
                if x_coord % shape[1] != 0 and y_coord % shape[0] != 0:
                    if len(k) == 3:
                        conf = k[2]
                        if conf < 0.5:
                            continue
                    cv2.circle(self.im, (int(x_coord), int(y_coord)),
                               int(radius * plot_size_redio), color_k, -1, lineType=cv2.LINE_AA)

            if kpt_line:
                ndim = kpts.shape[-1]
                for i, sk in enumerate(self.skeleton):
                    if show_skeleton is not None:
                        if sk not in show_skeleton:
                            continue
                    pos1 = (int(kpts[(sk[0] - 1), 0]), int(kpts[(sk[0] - 1), 1]))
                    pos2 = (int(kpts[(sk[1] - 1), 0]), int(kpts[(sk[1] - 1), 1]))
                    if ndim == 3:
                        conf1 = kpts[(sk[0] - 1), 2]
                        conf2 = kpts[(sk[1] - 1), 2]
                        if conf1 < 0.5 or conf2 < 0.5:
                            continue
                    if pos1[0] % shape[1] == 0 or pos1[1] % shape[0] == 0 or pos1[0] < 0 or pos1[1] < 0:
                        continue
                    if pos2[0] % shape[1] == 0 or pos2[1] % shape[0] == 0 or pos2[0] < 0 or pos2[1] < 0:
                        continue
                    cv2.line(self.im, pos1, pos2, [int(x) for x in self.limb_color[i]],
                             thickness=int(line_thickness * plot_size_redio), lineType=cv2.LINE_AA)
            if self.pil:
                # Convert im back to PIL and update draw
                self.fromarray(self.im)

    annotator = _Annotator(deepcopy(pose_result.orig_img))
    if pose_result.keypoints is not None:
        for k in reversed(pose_result.keypoints.data):
            annotator.kpts(k, pose_result.orig_shape, kpt_line=True)
    return annotator.result()


def put_text(frame, exercise, count, fps, redio):

    if exercise in sport_list.keys():
        cv2.putText(
            frame, f'Exercise: {exercise}', (int(10 * redio), int(30 * redio)), 0, 0.7 * redio,
            (255, 255, 255), thickness=int(2 * redio), lineType=cv2.LINE_AA
        )
    elif exercise == 'No Object':
        cv2.putText(
            frame, f'No Object', (int(10 * redio), int(30 * redio)), 0, 0.7 * redio,
            (255, 255, 255), thickness=int(2 * redio), lineType=cv2.LINE_AA
        )
    cv2.putText(
        frame, f'Count: {count}', (int(10 * redio), int(60 * redio)), 0, 0.7 * redio,
        (255, 255, 255), thickness=int(2 * redio), lineType=cv2.LINE_AA
    )
    cv2.putText(
        frame, f'FPS: {fps}', (int(10 * redio), int(90 * redio)), 0, 0.7 * redio,
        (255, 255, 255), thickness=int(2 * redio), lineType=cv2.LINE_AA
    )

def main(value,name):
    model = YOLO('yolov8n-pose.pt') 
    cap = cv2.VideoCapture(name)
    count = 0
    prev_angle = None
    start_counting = False
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            plot_size_ratio = max(frame.shape[1] / 960, frame.shape[0] / 540)

            results = model(frame,verbose=False)
            if results[0].keypoints.shape[1] == 0:
                continue
            if value=="squat":
                left_points_idx = sport_list['squat']['left_points_idx']
                right_points_idx = sport_list['squat']['right_points_idx']
            elif value=='deadlift':
                left_points_idx = sport_list['deadlift']['left_points_idx']
                right_points_idx = sport_list['deadlift']['right_points_idx']
            else:
                left_points_idx = sport_list['bench']['left_points_idx']
                right_points_idx = sport_list['bench']['right_points_idx']
            angle = calculate_angle(results[0].keypoints, left_points_idx, right_points_idx)

            if prev_angle is not None:
                print(sport_list[value]['maintaining'])
                print(angle)
                print(sport_list[value]['relaxing'])
                if angle < sport_list[value]['maintaining'] and not start_counting:

                    start_counting = True
                    relax_count=0
                elif angle > sport_list[value]['relaxing'] and start_counting:
                    if value=="bench":
                        relax_count=relax_count+1
                    if relax_count==2:
                        count += 1
                        start_counting = False

            prev_angle = angle

            annotated_frame = plot(results[0], plot_size_ratio)
            put_text(annotated_frame, value, count, 0, plot_size_ratio)
            cv2.imshow("YOLOv8 Inference", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    video="bench.MOV"
    main(value="bench",name=video)


70
37.41107776456765
110
70
27.48375941675711
110
70
28.417559772117528
110
70
24.384804382876396
110
70
48.80572604964249
110
70
56.59886724693236
110
70
36.72133008407001
110
70
48.68329643141937
110
70
47.61493092382626
110
70
36.771388403730455
110
70
17.1538814024479
110
70
11.610534691956751
110
70
18.979125879549883
110
70
21.222753410249865
110
70
11.034850597711937
110
70
9.726488729646796
110
70
72.32666102698184
110
70
48.93973224693008
110
70
65.3233336061424
110
70
77.20324937136716
110
70
54.066695563505874
110
70
42.36535321187528
110
70
70.74575097337957
110
70
95.62396169659937
110
70
56.0702145259703
110
70
51.599775763528356
110
70
48.26818242324933
110
70
40.819122764631715
110
70
53.64605451518839
110
70
59.88413273117338
110
70
65.61885993944183
110
70
110.07354282672028
110
70
128.93925260952477
110
70
67.58709702196293
110
70
90.76050251125744
110
70
75.8880094040342
110
70
35.76176120977584
110
70
30.037784960875
110
70
32.90813810186718
110
70
57.0175043035380